In [62]:
import requests
from bs4 import BeautifulSoup
import json
import time
from datetime import datetime

from requests.adapters import HTTPAdapter
from urllib3.util import Retry
# Liste des URLs Gaza fournie plus haut
urls_gaza = [ 
        "https://www.foxnews.com/world/israeli-military-sees-hamas-war-lasting-all-2024-report"
        ,"https://www.theguardian.com/world/2025/jan/15/the-devastating-impact-of-15-months-of-war-on-gaza"
        ,"https://www.foxnews.com/world/idf-seized-weapons-cache-supplies-hamas-terror-groups-attack-israel"
        ,"https://www.foxnews.com/world/israeli-defense-forces-release-video-showing-evidence-hamas-weapons-tunnels-linking-hospital-basements"
        ,"https://www.foxnews.com/world/israeli-troops-enter-al-shifa-hospital-killing-hamas-terrorists-gate-side-firefight"
        ,"https://www.foxnews.com/world/urban-warfare-expert-says-israeli-military-taking-unprecedented-steps-to-protect-gaza-civilians"
        ,"https://www.foxnews.com/world/idf-investigating-whether-hamas-chief-yahya-sinwar-killed-gaza-operations"     
        ,"https://www.foxnews.com/world/hamas-command-tunnel-unrwas-headquarters-gaza-israel-says"
        ,"https://www.foxnews.com/politics/ex-hamas-hostage-freed-after-505-days-gaza-tunnels-warns-evil-spreading-across-world"
        ,"https://www.foxnews.com/world/anti-hamas-protests-breakout-gaza-strip-over-demands-end-war"
        
        ,"https://www.foxnews.com/politics/trump-warns-hamas-war-resume-terror-group-continues-kill-people-gaza"
        ,"https://www.foxnews.com/world/after-trump-declares-war-over-hamas-executes-rivals-gaza-reassert-control"
        ,"https://www.foxnews.com/world/israeli-military-sees-hamas-war-lasting-all-2024-report"
        ,"https://www.foxnews.com/world/israel-says-hamas-violated-ceasefire-multiple-attacks-leading-idf-response"
        ,"https://www.theguardian.com/world/2025/oct/07/diplomacy-lowest-point-how-the-israel-gaza-conflict-was-mishandled"
        ,"https://www.foxnews.com/world/study-disputes-gaza-genocide-charges-finds-flawed-data-amid-hamas-driven-narrative"
        ,"https://www.foxnews.com/world/ceasefire-over-israel-strikes-gaza-after-hamas-refused-release-hostages-officials-say"
        ,"https://www.foxnews.com/world/israels-government-likely-approve-plan-re-occupy-gaza-israeli-media-reports?msockid=1cd7b05cd5a66a1e1db0a6c1d4e66b41"
        ,"https://www.foxnews.com/world/after-trump-declares-war-over-hamas-executes-rivals-gaza-reassert-control?msockid=1cd7b05cd5a66a1e1db0a6c1d4e66b41"
        ,"https://www.foxnews.com/world/arab-nations-first-time-pressure-hamas-disarm-relinquish-power-gaza?msockid=1cd7b05cd5a66a1e1db0a6c1d4e66b41"
        
        ,"https://www.foxnews.com/world/israels-evacuation-orders-cover-more-than-two-thirds-gaza-strip-war-hamas-continues?msockid=1cd7b05cd5a66a1e1db0a6c1d4e66b41"
        ,"https://www.foxnews.com/world/israel-hamas-terrorists-reoccurring-battles-around-gaza-strip?msockid=1cd7b05cd5a66a1e1db0a6c1d4e66b41"
        ,"https://bbc.com/news/articles/clyv7w3gdy2o"
        ,"https://bbc.com/news/world-middle-east-67480680"
        # ,"https://bbc.com/news/world-middle-east-6747724"
        ,"https://bbc.com/news/articles/c4gdx1rpel6o"
        ,"https://bbc.com/news/articles/cx2dklj3rxvo"
        ,"https://www.cnn.com/2023/12/21/middleeast/palestinians-back-hamas-survey-intl-cmd"
        ,"https://www.foxnews.com/politics/ex-hamas-hostage-freed-after-505-days-gaza-tunnels-warns-evil-spreading-across-world"
        ,"https://www.foxnews.com/world/taking-out-hamas-million-dollar-root-tunnel-game-changer-analyst-says"
        
        ,"https://www.foxnews.com/world/hamas-hardens-stance-ceasefire-talks-netanyahu-recalls-negotiation-team"
        ,"https://www.foxnews.com/world/former-hamas-hostage-warned-australian-leaders-about-dangers-antisemitism-months-before-bondi-beach-attack"
        ,"https://www.foxnews.com/world/hamas-faces-legitimacy-crisis-desperate-gazans-flock-us-backed-aid-centers",
        # --- FOX NEWS (Focus: Terrorisme et Menace) ---
        "https://www.foxnews.com/world/israel-constructs-large-pumps-flood-hamas-terror-tunnels-seawater-wall-street-journal",
        # "https://www.foxnews.com/world/israel-eliminates-top-hamas-commander-behind-deadly-october-7-attacks",
        # "https://www.foxnews.com/world/idf-video-captures-hamas-terrorists-using-ambulance-for-transport-gaza",
        # "https://www.foxnews.com/world/hamas-terrorists-intercept-humanitarian-aid-trucks-gaza-video",
        # "https://www.foxnews.com/world/israel-claims-hamas-stashed-weapons-under-infant-crib-gaza-hospital",
        # "https://www.foxnews.com/opinion/israel-moral-clarity-hamas-barbarity-war-civilization",
        
        # "https://www.foxnews.com/world/israeli-special-forces-rescue-hostages-daring-daylight-raid-gaza",
        # --- BBC NEWS (Focus: Neutralité technique / Faute d'attribution) ---
        # "https://www.bbc.com/news/world-middle-east-68910000", # Explosion at hospital (passive voice)
        # "https://www.bbc.com/news/world-middle-east-67330000", # Gaza health ministry says (distanciation)
        "https://www.bbc.com/news/articles/c17x0xjqj21o", # Body of hostage returned (clinical tone)
        # "https://www.bbc.com/news/world-middle-east-68120000", # Temporary pier for aid (technical focus)
        # --- CNN & WSJ (Focus: Ambiguïté et Biais Institutionnel) ---
        # "https://www.cnn.com/2024/05/22/middleeast/israel-gaza-war-journalism-bias-intl",
        # "https://www.cnn.com/2023/11/15/middleeast/gaza-al-shifa-hospital-idf-intl",
        # "https://www.wsj.com/world/middle-east/hamas-fighters-trained-in-iran-before-oct-7-attacks-e794ad7a",
        # "https://www.wsj.com/world/middle-east/israel-says-hamas-uses-al-shifa-hospital-as-command-center-11603847253",
        # "https://www.theguardian.com/world/2024/feb/04/cnn-staff-divided-over-network-pro-israel-editorial-policy"
        ] 
corpus_gaza = []

# --- 1. MISE EN PLACE D'UNE STRATÉGIE DE RETRY ---
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
print("--- Début de la collecte du corpus gaza ---")
session = requests.Session()
retry_strategy = Retry(
    total=3, # Réessaye 3 fois max
    backoff_factor=2, # Attend 2s, 4s, 8s entre les essais
    status_forcelist=[429, 500, 502, 503, 504] # Codes d'erreurs serveurs classiques
)
# --- 2. BOUCLE MODIFIÉE ---
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)
# --- 2. BOUCLE MODIFIÉE ---
for url in urls_gaza:

    try:

        # On utilise session.get au lieu de requests.get
        # On augmente le timeout à (connect_timeout, read_timeout)
        r = session.get(url, headers=headers, timeout=(5, 30))   
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            
            # --- 1. Titre ---
            title_tag = soup.find('h1')
            titre = title_tag.get_text(strip=True) if title_tag else "Sans titre"
            
            # --- 2. Détection Source & Nettoyage spécifique ---            source = "BBC"
            source = "Autre"
            author = "N/A"
            publish_date = "N/A"
            article_body = None            
            
            
            if "foxnews.com" in url:
                source = "Fox News"
                # Cible le corps de l'article chez Fox
                article_body = soup.find('div', class_='article-body')
                # Cherche l'auteur
                author_tag = soup.find('div', class_='author-byline')
                if author_tag: author = author_tag.get_text(strip=True).replace('By ', '')
                # Cherche la date
                date_tag = soup.find('time')
                if date_tag: publish_date = date_tag.get_text(strip=True)

            elif "bbc.com" in url:
                source = "BBC"
                article_body = soup.find('article')
                date_tag = soup.find('time')
                if date_tag: publish_date = date_tag.get('datetime')
            
            
            elif "theguardian.com" in url:
                source = "The Guardian"
                article_body = soup.find('div', class_='article-body-commercial-selector')
                author_tag = soup.find('a', rel='author')
                if author_tag: author = author_tag.get_text(strip=True)
                date_tag = soup.find('time')
                if date_tag: publish_date = date_tag.get('datetime')
             

            # --- 3. Extraction du texte (uniquement si on a trouvé le corps) ---

            # Extraction des paragraphes avec filtre
            # On ignore les textes trop courts pour éviter les menus/pubs

            if article_body:
                paragraphes = [p.get_text(strip=True) for p in article_body.find_all('p') if len(p.get_text()) > 25 and "©" not in p.get_text()]
            else:
                # Fallback si la structure est différente
                paragraphes = [p.get_text(strip=True) for p in soup.find_all('p') 
                            if len(p.get_text()) > 40 and "©" not in p.get_text()]

            texte_final = "\n".join(paragraphes)
            # On n'ajoute l'article que s'il y a du contenu            
            if len(texte_final) > 100:
                corpus_gaza.append({
                    "source": source,
                    "url": url,
                    "title": titre,
                    "publish_date": publish_date,
                    "author": author,
                    "content": texte_final,
                    "scraped_at": datetime.now().isoformat(),
                    "keywords": [],
                    "summary": "",
                    "conflict": "Gaza"
                })
                print(f"Succès : {titre[:50]}...")
            else:
                print(f"Contenu trop court ou vide pour : {url}")
        else:
            print(f" Erreur {r.status_code} pour l'URL : {url}")

        # Pause éthique pour respecter le serveur
        time.sleep(3) 

    except Exception as e:
        print(f"Erreur critique sur  {url}: {e}")

# 2. Sauvegarde dans un fichier JSON
file_name = 'corpus_gaza.json'
with open('corpus_gaza.json', 'w', encoding='utf-8') as f:
    json.dump(corpus_gaza, f, indent=4, ensure_ascii=False)
print(f"\n--- TERMINÉ ---")
print(f"Résultat : {len(corpus_gaza)} articles sauvegardés dans {file_name}")

# Téléchargement automatique (Si  utilisez Google Colab)
try:
    from google.colab import files
    files.download(file_name)
except ImportError:
    print(f"Note : Le fichier est disponible localement sous le nom {file_name}")


--- Début de la collecte du corpus gaza ---
Succès : Israeli military sees Hamas war lasting through al...
Succès : The devastating impact of 15 months of war on Gaza...
Succès : IDF seized weapons cache, supplies from Hamas duri...
Succès : Israeli Defense Forces release video showing evide...
Succès : Israeli troops enter Al-Shifa Hospital after killi...
Succès : Urban warfare expert says Israeli military taking ...
Succès : Hamas terror leader Yahya Sinwar killed in Gaza, I...
Succès : Hamas had command tunnel underneath UNRWA’s headqu...
Succès : Ex-Hamas hostage freed after 505 days in Gaza tunn...
Succès : Anti-Hamas protests break out in Gaza Strip over d...
Succès : Trump warns Hamas if terror group 'continues to ki...
Succès : After Trump declares ‘war is over,’ Hamas executes...
Succès : Israeli military sees Hamas war lasting through al...
Succès : Israel says Hamas violated ceasefire with 'multipl...
Succès : Diplomacy’s lowest point: how the Israel-Gaza conf...
Succès : St